In [1]:
from collections import namedtuple
import scipy.stats as sps
import statsmodels.stats.api as sms
from tqdm.notebook import tqdm as tqdm_notebook
from collections import defaultdict
from statsmodels.stats.proportion import proportion_confint
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, palette='Set2')
ExperimentComparisonResults = namedtuple('ExperimentComparisonResults', 
                                        ['pvalue', 'effect', 'ci_length', 'left_bound', 'right_bound'])

# Примеры спаренных критериев 

**Absolute t-test**

In [12]:
def absolute_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)
    
    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [2]:
def paired_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    
    difference_mean = mean_test - mean_control
    difference_mean_var = np.var(test - control) / len(control)
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

**Relative t-test**

In [3]:
def paired_relative_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    
    var_mean_control = np.var(control) / len(control)
    difference_mean = mean_test - mean_control
    difference_mean_var  = np.var(test - control) / len(test)
    
    covariance = -np.cov(test - control, control)[0, 1] / len(control)

    relative_mu = difference_mean / mean_control
    relative_var = difference_mean_var / (mean_control ** 2) \
                    + var_mean_control * ((difference_mean ** 2) / (mean_control ** 4))\
                    - 2 * (difference_mean / (mean_control ** 3)) * covariance
    relative_distribution = sps.norm(loc=relative_mu, scale=np.sqrt(relative_var))
    left_bound, right_bound = relative_distribution.ppf([0.025, 0.975])
    
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(relative_distribution.cdf(0), relative_distribution.sf(0))
    effect = relative_mu
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

**Relative cuped**

In [4]:
def paired_relative_cuped(control, test, control_before, test_before):
    theta = np.cov(test - control, test_before - control_before)[0, 1] /\
                np.var(test_before - control_before)

    control_cup = control - theta * control_before
    test_cup = test - theta * test_before

    mean_den = np.mean(control)
    mean_num = np.mean(test_cup - control_cup)
    var_mean_den  = np.var(control) / len(control)
    var_mean_num  = np.var(test_cup - control_cup) / len(control_cup)

    cov = -np.cov(test_cup - control_cup, control)[0, 1] / len(control)

    relative_mu = mean_num / mean_den
    relative_var = var_mean_num / (mean_den ** 2)  + var_mean_den * ((mean_num ** 2) / (mean_den ** 4))\
                - 2 * (mean_num / (mean_den ** 3)) * cov
    
    relative_distribution = sps.norm(loc=relative_mu, scale=np.sqrt(relative_var))
    left_bound, right_bound = relative_distribution.ppf([0.025, 0.975])
    
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(relative_distribution.cdf(0), relative_distribution.sf(0))
    effect = relative_mu
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

**Парная пост-нормировка**

In [5]:
def paired_post_normed_bootstrap(control, test, control_before, test_before, test_type='absolute'):
    # Функция от средних, которую надо посчитать на каждой выборке
    absolute_func = lambda C, T, C_b, T_b: T - (T_b / C_b) * C
    relative_func = lambda C, T, C_b, T_b: (T / C) / (T_b / C_b) - 1
    
    boot_func = absolute_func if test_type == 'absolute' else relative_func
    stat_sample = []
    
    batch_sz = 100
    
    #В теории boot_samples_size стоить брать не меньше размера выборки. Но на практике можно и меньше.
    boot_samples_size = len(control)
    for i in range(0, boot_samples_size, batch_sz):
        N = len(control)

        # Надо помнить, что мы семплируем именно юзеров
        # Поэтому, если мы взяли n раз i элемент в выборке control
        # То надо столько же раз взять i элемент в выборке control_before
        # Поэтому будем семплировать индексы
        indices = np.arange(N)

        indices_sample = np.random.choice(control_indices, size=(len(control), batch_sz), replace=True)

        
        C   = np.mean(control[indices_sample], axis=0)
        T   = np.mean(test[indices_sample], axis=0)
        C_b = np.mean(control_before[indices_sample], axis=0)
        T_b = np.mean(test_before[indices_sample], axis=0)
        assert len(T) == batch_sz
        stat_sample += list(boot_func(C, T, C_b, T_b))

    stat_sample = np.array(stat_sample)
    # считаем истинный эффект
    effect = boot_func(np.mean(control), np.mean(test), np.mean(control_before), np.mean(test_before))
    left_bound, right_bound = np.quantile(stat_sample, [0.025, 0.975])
    
    ci_length = (right_bound - left_bound)
    # pvalue - процент статистик, которые лежат левее или правее 0.
    pvalue = 2 * min(np.mean(stat_sample > 0), np.mean(stat_sample < 0))
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

# Алгоритм сплитования в парной стратификации

In [6]:
def splitter(before_metrics):
    size = len(before_metrics)
    
    # отсортируем массив 
    sorted_array = np.sort(before_metrics)[::-1]
    control = []
    test = []
    for i in range(0, size, 2):
        if np.random.rand() < 0.5:
            control.append(sorted_array[i])
            test.append(sorted_array[i + 1])
        else:
            control.append(sorted_array[i + 1])
            test.append(sorted_array[i])

    return np.array(control), np.array(test)

# сравнение мощностей

### парная стратификация применена

In [7]:
def relative_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)

    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    left_bound = left_bound / np.mean(control)   # Деление на среднее
    right_bound = right_bound / np.mean(control) # Деление на среднее

    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [8]:
bad_cnt_paired = 0
paired_power = 0
bad_cnt = 0
power = 0
N = 30000
for i in tqdm_notebook(range(N)):
    before = sps.expon(scale=1000).rvs(4000)
    C_b, T_b = splitter(before)

    C = C_b + sps.norm(loc=0, scale=100).rvs(2000)
    T = T_b + sps.norm(loc=0, scale=100).rvs(2000)
    T *= 1.01

    _, _, _, left_bound, right_bound = relative_ttest(C, T)
    _, _, _, left_bound_paired, right_bound_paired = paired_relative_ttest(C, T)

    if left_bound > 0.01 or right_bound < 0.01:
        bad_cnt += 1
        
    if left_bound_paired > 0.01 or right_bound_paired < 0.01:
        bad_cnt_paired += 1

    if left_bound > 0:
        power += 1

    if left_bound_paired > 0:
        paired_power += 1
    
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, 
                                                         alpha=0.05, method='wilson')
left_real_level_paired, right_real_level_paired = proportion_confint(count = bad_cnt_paired, nobs = N, 
                                                         alpha=0.05, method='wilson')
print(f"Реальный уровень значимости для обычного критрия: {round(bad_cnt / N, 4)};"\
      f" доверительный интервал: [{round(left_real_level, 5)}, {round(right_real_level, 5)}]")
print(f"Реальный уровень значимости для спаренного критерия: {round(bad_cnt_paired / N, 4)};"\
      f" доверительный интервал: [{round(left_real_level_paired, 5)}, {round(right_real_level_paired, 5)}]")
print(f"Мощность спаренного критрерия VS мощность обычного критерия: {paired_power / N} VS. {power / N}")

  0%|          | 0/30000 [00:00<?, ?it/s]

Реальный уровень значимости для обычного критрия: 0.0; доверительный интервал: [0.0, 0.00013]
Реальный уровень значимости для спаренного критерия: 0.0513; доверительный интервал: [0.04886, 0.05385]
Мощность спаренного критрерия VS мощность обычного критерия: 0.8623 VS. 0.0


### стратификации не было

In [13]:
bad_cnt_paired = 0
paired_power = 0
bad_cnt = 0
power = 0
N = 30000
for i in tqdm_notebook(range(N)):
    before = sps.expon(scale=1000).rvs(4000)
    C_b, T_b = before[:2000], before[2000:]

    C = C_b + sps.norm(loc=0, scale=100).rvs(2000)
    T = T_b + sps.norm(loc=0, scale=100).rvs(2000)
    T *= 1.01

    _, _, _, left_bound, right_bound = relative_ttest(C, T)
    _, _, _, left_bound_paired, right_bound_paired = paired_relative_ttest(C, T)

    if left_bound > 0.01 or right_bound < 0.01:
        bad_cnt += 1
        
    if left_bound_paired > 0.01 or right_bound_paired < 0.01:
        bad_cnt_paired += 1

    if left_bound > 0:
        power += 1

    if left_bound_paired > 0:
        paired_power += 1
    
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, 
                                                         alpha=0.05, method='wilson')
left_real_level_paired, right_real_level_paired = proportion_confint(count = bad_cnt_paired, nobs = N, 
                                                         alpha=0.05, method='wilson')
print(f"Реальный уровень значимости для обычного критрия: {round(bad_cnt / N, 4)};"\
      f" доверительный интервал: [{round(left_real_level, 5)}, {round(right_real_level, 5)}]")
print(f"Реальный уровень значимости для спаренного критерия: {round(bad_cnt_paired / N, 4)};"\
      f" доверительный интервал: [{round(left_real_level_paired, 5)}, {round(right_real_level_paired, 5)}]")
print(f"Мощность спаренного критрерия VS мощность обычного критерия: {paired_power / N} VS. {power /N}")

  0%|          | 0/30000 [00:00<?, ?it/s]

Реальный уровень значимости для обычного критрия: 0.0505; доверительный интервал: [0.04808, 0.05304]
Реальный уровень значимости для спаренного критерия: 0.0507; доверительный интервал: [0.04824, 0.05321]
Мощность спаренного критрерия VS мощность обычного критерия: 0.05503333333333333 VS. 0.0482


## Пример, когда от сплитования может стать хуже

In [14]:
ci_NOT_splitted_length = []
ci_splitted_length = []
N = 1000
# сплитование
for i in tqdm_notebook(range(N)):
    before = sps.expon(scale=1000).rvs(4000)
    # уже отсортированы
    C_b, T_b = splitter(before)

    C = C_b
    T = -1 * T_b

    _, _, ci_splitted_sample, _, _ = paired_ttest(C, T)
    ci_splitted_length.append(ci_splitted_sample)
###################################
    
# не было сплитования
for i in tqdm_notebook(range(N)):
    before = sps.expon(scale=1000).rvs(4000)
    C_b, T_b = before[:2000], before[2000:]

    C = C_b
    T = -1 * T_b

    _, _, ci_NOT_splitted_sample, _, _ = absolute_ttest(C, T)

    ci_NOT_splitted_length.append(ci_NOT_splitted_sample)
print("ширина доверительного интервала в случае неспаренного критерия VS спаренного критерия:"\
      f" {round(np.mean(ci_NOT_splitted_length), 2)} VS. {round(np.mean(ci_splitted_length), 2)}")
print(f"pvalue сравнения: {absolute_ttest(np.array(ci_NOT_splitted_length), np.array(ci_splitted_length)).pvalue}")

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

ширина доверительного интервала в случае неспаренного критерия VS спаренного критерия: 123.8 VS. 174.96
pvalue сравнения: 0.0
